# Lesson 1: Building blocks

Welcome to this course! Just a reminder that LLMs may not always return the same text. As a result, the outputs you see when running the notebook may vary somewhat from those you see in the film. This applies to all of the notebooks.
Enjoy the course!

## Language model

1. Text LLMs: string -> string
2. Chat models: list of messages -> single message output

In [1]:
import "dotenv/config";

[Module: null prototype] { default: {} }

In [2]:
import { ChatOpenAI } from "@langchain/openai";
import { HumanMessage } from "@langchain/core/messages";

In [3]:
const model = new ChatOpenAI({
    modelName: "gpt-3.5-turbo-1106"
});

await model.invoke([
    new HumanMessage("Tell me a joke.")
]);

AIMessage {
  lc_serializable: true,
  lc_kwargs: {
    content: "Why don't skeletons fight each other? They don't have the guts.",
    additional_kwargs: { function_call: undefined, tool_calls: undefined }
  },
  lc_namespace: [ "langchain_core", "messages" ],
  content: "Why don't skeletons fight each other? They don't have the guts.",
  name: undefined,
  additional_kwargs: { function_call: undefined, tool_calls: undefined }
}

## Prompt template

In [4]:
import { ChatPromptTemplate } from "@langchain/core/prompts";

const prompt = ChatPromptTemplate.fromTemplate(
    `What are three good names for a company that makes {product}?`
)

In [5]:
await prompt.format({
    product: "colorful socks"
});

"Human: What are three good names for a company that makes colorful socks?"

In [6]:
await prompt.formatMessages({
    product: "colorful socks"
});

[
  HumanMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "What are three good names for a company that makes colorful socks?",
      additional_kwargs: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "What are three good names for a company that makes colorful socks?",
    name: undefined,
    additional_kwargs: {}
  }
]

In [7]:
import { 
  SystemMessagePromptTemplate, 
  HumanMessagePromptTemplate 
} from "@langchain/core/prompts";

const promptFromMessages = ChatPromptTemplate.fromMessages([
  SystemMessagePromptTemplate.fromTemplate(
    "You are an expert at picking company names."
  ),
  HumanMessagePromptTemplate.fromTemplate(
    "What are three good names for a company that makes {product}?"
  )
]);

await promptFromMessages.formatMessages({
    product: "shiny objects"
});

[
  SystemMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "You are an expert at picking company names.",
      additional_kwargs: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "You are an expert at picking company names.",
    name: undefined,
    additional_kwargs: {}
  },
  HumanMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "What are three good names for a company that makes shiny objects?",
      additional_kwargs: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "What are three good names for a company that makes shiny objects?",
    name: undefined,
    additional_kwargs: {}
  }
]

In [8]:
const promptFromMessages = ChatPromptTemplate.fromMessages([
  ["system", "You are an expert at picking company names."],
  ["human", "What are three good names for a company that makes {product}?"]
]);

await promptFromMessages.formatMessages({
  product: "shiny objects"
});

[
  SystemMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "You are an expert at picking company names.",
      additional_kwargs: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "You are an expert at picking company names.",
    name: undefined,
    additional_kwargs: {}
  },
  HumanMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "What are three good names for a company that makes shiny objects?",
      additional_kwargs: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "What are three good names for a company that makes shiny objects?",
    name: undefined,
    additional_kwargs: {}
  }
]

# LangChain Expression Language (LCEL)

In [9]:
const chain = prompt.pipe(model);

In [10]:
await chain.invoke({
    product: "colorful socks"
});

AIMessage {
  lc_serializable: true,
  lc_kwargs: {
    content: "1. Brightfeet\n2. Rainbowsoles\n3. ColorfulStride",
    additional_kwargs: { function_call: undefined, tool_calls: undefined }
  },
  lc_namespace: [ "langchain_core", "messages" ],
  content: "1. Brightfeet\n2. Rainbowsoles\n3. ColorfulStride",
  name: undefined,
  additional_kwargs: { function_call: undefined, tool_calls: undefined }
}

## Output parser

In [11]:
import { StringOutputParser } from "@langchain/core/output_parsers";

const outputParser = new StringOutputParser();

const nameGenerationChain = prompt.pipe(model).pipe(outputParser);

In [12]:
await nameGenerationChain.invoke({
    product: "fancy cookies"
});

"1. Decadent Delights\n2. Gourmet Elegance\n3. Sweet Indulgences Co."

https://smith.langchain.com/public/9fd7689f-23c6-4ea1-8a76-fea1b5f8d5db/r

In [15]:
import { RunnableSequence } from "@langchain/core/runnables";

const nameGenerationChain = RunnableSequence.from([
    prompt,
    model,
    outputParser
])

In [16]:
await nameGenerationChain.invoke({
    product: "fancy cookies"
});

"1. Decadent Delights\n2. Gourmet Elegance\n3. Sweet Indulgences Co."

## Streaming

In [17]:
const stream = await nameGenerationChain.stream({
  product: "really cool robots",
});

for await (const chunk of stream) {
    console.log(chunk);
}


1
.
 Rob
o
Tech
 Innov
ations


2
.
 Mech
Master
 Robotics


3
.
 Future
B
ots
 Unlimited



## Batch

In [18]:
const inputs = [
    { product: "large calculators" },
    { product: "alpaca wool sweaters" }
];

await nameGenerationChain.batch(inputs);

[
  "1. Titan Calc\n2. MegaMath Co.\n3. BigDigits Inc.",
  "1. Alpaca Threads\n2. Andean Luxe\n3. Wooly Alpaca Co."
]